In [1]:
from CT_COVID_Preprocess import CTCOVIDDataset
from model_tools import validation, train, load_model_checkpoint
from eval_tools import VGG19_CAM
from eval_tools import plot_train_val, test, display_FP_FN, get_confusion_matrix, plot_ROCAUC_curve, showCAM
from custom_model import CNN_COVIDCT
import torch
import torchvision
import torch.optim as optim
from monai.transforms import Compose, LoadPNG, AddChannel, ScaleIntensity, ToTensor, RandRotate, RandFlip, RandZoom, Resize, RandGaussianNoise
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader
from collections import OrderedDict
from torchvision import transforms

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
dirCOVID = 'C:/Users/rrsoo/AdvancedStuff/Medical_AI/Classifier/Images-processed-new/CT_COVID'
dirNonCOVID = 'C:/Users/rrsoo/AdvancedStuff/Medical_AI/Classifier/Images-processed-new/CT_NonCOVID'

In [4]:
train_transforms = transforms.Compose([
        LoadPNG(),
        AddChannel(),
        ScaleIntensity(),
        RandRotate(degrees=15, prob=0.5),
        RandFlip(spatial_axis=0, prob=0.5),
        RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
        RandGaussianNoise(prob = 0.5),
        Resize(spatial_size=(224, 224)),
        ToTensor()
    ])
   

val_transforms = transforms.Compose([
    LoadPNG(),
    AddChannel(),
    ScaleIntensity(),
    ToTensor()
])

In [5]:
BATCH_SIZE = 32
ORIG_RES = False

train_ds = CTCOVIDDataset(dirCOVID, dirNonCOVID, transforms = train_transforms, data = 'train', orig_res = ORIG_RES)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

val_ds = CTCOVIDDataset(dirCOVID, dirNonCOVID, transforms = val_transforms, data = 'val', orig_res = ORIG_RES)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True)

test_ds = CTCOVIDDataset(dirCOVID, dirNonCOVID, transforms = val_transforms, data = 'test', orig_res = ORIG_RES)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)

100%|██████████| 346/346 [00:01<00:00, 324.84it/s]


In [6]:
model_name = 'resnet34' #changeable
PRETRAINED = True #changeable

model = None #Don't touch

if model_name == 'resnet34':
    model = torchvision.models.resnet34(pretrained = PRETRAINED).to(device)
    
    if PRETRAINED:
        for param in model.parameters():
            param.requires_grad = False
            
    layers_resnet = nn.Sequential(OrderedDict([
                ('fc1', nn.Linear(512, 256)),
                ('activation1', nn.ReLU()),
                ('fc2', nn.Linear(256, 128)),
                ('activation2', nn.ReLU()),
                ('fc3', nn.Linear(128, 2)),
                ('out', nn.Sigmoid())
            ])).to(device)
    
    model.fc = layers_resnet

assert(model is not None)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
epochs = 40
optimizer = optim.Adam(model.parameters())
#scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-3, max_lr=1e-2, cycle_momentum=False)
criterion = nn.CrossEntropyLoss()
history = train(model, train_loader, val_loader, optimizer, criterion, epochs, device = device, scheduler = None)

  5%|▌         | 1/19 [00:00<00:02,  6.57it/s]

Training Loss: 13.034444749355316
Training Accuracy: 0.5260504201680672
Validation Loss: 2.0720869302749634
Validation Accuracy: 0.527027027027027


  5%|▌         | 1/19 [00:00<00:02,  6.60it/s]

Training Loss: 12.409207582473755
Training Accuracy: 0.6033613445378151
Validation Loss: 1.969240963459015
Validation Accuracy: 0.6081081081081081


  5%|▌         | 1/19 [00:00<00:02,  6.55it/s]

...

Training Loss: 7.060862749814987
Training Accuracy: 0.9428571428571428
Validation Loss: 1.2645426988601685
Validation Accuracy: 0.9324324324324325


  5%|▌         | 1/19 [00:00<00:02,  6.55it/s]

Training Loss: 7.155577540397644
Training Accuracy: 0.9378151260504202
Validation Loss: 1.228758990764618
Validation Accuracy: 0.9324324324324325


100%|██████████| 3/3 [00:00<00:00,  8.69it/s]

Training Loss: 7.732251077890396
Training Accuracy: 0.9008403361344538
Validation Loss: 1.224666714668274
Validation Accuracy: 0.918918918918919


In [2]:
plot_train_val(history, epochs, (10, 10))

NameError: name 'plot_train_val' is not defined

In [1]:
model_dir = 'resnetforpaper.pth'

checkpoints = {
    'model' : model,
    'input_shape' : (224, 224, 3),
    'batch_size' : 32,
    'epochs' : 40,
    'train_acc' : 91,
    'val_acc' : 90,
    'state_dict' : model.state_dict()
}

torch.save(checkpoints, model_dir)

NameError: name 'model' is not defined